In [1]:
# %%writefile alpha_base10.py

import numpy as np
import pandas as pd
from scipy import stats
from sklearn import preprocessing
from sklearn import linear_model

def STD(data, windows):
    return data.rolling(window=windows, min_periods=windows).std()
def MEAN(data, windows):
    return data.rolling(window=windows, min_periods=windows).mean()
def DELTA(data, windows):
    return data.diff(4)
def SEQUENCE(n):
    return pd.Series(np.arange(1,n+1))

def SMA(data,windows,alpha):
    return data.ewm(adjust=False, alpha=float(alpha)/windows, min_periods=windows, ignore_na=False).mean()

def REGBETA(xs, y, n):
    assert len(y)>=n,  'len(y)!>=n !!!'+ str(y.index[0])
    regress = linear_model.LinearRegression(fit_intercept=False)
    def reg(X,Y):
        try:
            if len(Y)>len(X):
                Y_ =  Y[X.index]
                if Y_.isnull().any():
                    return np.nan
                res = regress.fit(X.values.reshape(-1, 1), Y_.values.reshape(-1, 1)).coef_[0]
            else:
                # if Y.isnull().any():
                #     return np.nan
                res = regress.fit(X.values.reshape(-1, 1), Y.values.reshape(-1, 1)).coef_[0]
        except Exception as e:
            print(e)
            return np.nan
        return res
    return xs.rolling(window=n, min_periods=n).apply(lambda x:reg(x,y))


def COVIANCE(A,B,d):
    se = pd.Series(np.arange(len(A.index)),index=A.index)
    se = se.rolling(5).apply(lambda x: A.iloc[x].cov(B.iloc[x]))
    return se

def CORR(A,B,d):
    se = pd.Series(np.arange(len(A.index)),index=A.index)
    se = se.rolling(5).apply(lambda x: A.iloc[x].corr(B.iloc[x]))
    return se

def excute_for_multidates(data, func, level=0, **pramas):
    return data.groupby(level=level, group_keys=False).apply(func,**pramas)



In [2]:
import sys
import os 
module_path = os.path.abspath(os.path.join('..')) 
if module_path not in sys.path: 
    sys.path.append(module_path)
    
from tools.Cacher import (CACHE_TYPE, load_caches_adv)
import tools.Sample_Tools as smpl
import tools.Pretreat_Tools as pretreat

%load_ext autoreload
%autoreload 2
%aimport tools.Cacher

In [4]:
codes = smpl.get_codes_by_market(sse='all',only_main=True,filter_st=True)
files = list(map(lambda x:x+'_train_qfq',codes))
files.extend(list(map(lambda x:x+'_tail_qfq',codes)))
# stock_df = load_caches_adv(files,cache_type=CACHE_TYPE.STOCK)

def generate_alpha_factors(fun_ids, df=None):
    import ind.alpha191 as a191
    from tools.Cacher import (CACHE_TYPE, save_cache,load_cache_adv,load_cache)
    from base.JuUnits import excute_for_multidates
    import numpy as np

    for i in fun_ids:
    # for i in ids:
        fun_name = 'alpha'+ '0'*(3- int(np.log10(i)+1)) +str(i)
        if fun_name != 'alpha030':
            indx = excute_for_multidates(df, lambda x: getattr(a191, fun_name)(x) ,level=1)
        else:
            indx = getattr(a191, fun_name)(df)
        # indx = excute_for_multidates(df_treated.loc[(slice(None),['000001','601728','000002','601825', '601868']),:], lambda x: getattr(a191, fun_name)(x) ,level=1)
        indx.name = '191_'+fun_name
        save_cache('{}_train'.format(indx.name), indx, cache_type=CACHE_TYPE.FACTOR)
        # print(indx)

worker=6
task = task_chunk_split(ids, worker)
results = parallal_task(worker, generate_alpha_factors, task, df=df_treated)

KeyboardInterrupt: 